In [ ]:
!kaggle datasets download -d maedemaftouni/large-covid19-ct-slice-dataset


: 

In [1]:
!pwd

/workspaces/Chest-Cancer-Classification/research


In [3]:
import os
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

# Define a data class to hold configuration details for data ingestion
@dataclass(frozen=True)
class DataIngestionConfig:
    # The root directory where all data ingestion artifacts will be stored
    root_dir: Path
    
    # The URL or identifier of the data source (e.g., a Kaggle dataset URL or identifier)
    source_URL: str
    
    # The local file path where the downloaded data file will be saved
    local_data_file: Path
    
    # The directory where the data file will be unzipped or extracted
    unzip_dir: Path

    


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        # Initialize the ConfigurationManager with file paths for config and parameters
        # Read the configuration and parameters from the specified file paths
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create the root directory specified in the configuration
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Extract the data ingestion configuration from the loaded config
        config = self.config.data_ingestion
        
        # Create the directory specified in the data ingestion config
        create_directories([config.root_dir])
        
        # Create and return a DataIngestionConfig object using the extracted data
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [7]:
import os
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import subprocess

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Download data from Kaggle using the Kaggle CLI.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)  # Ensure the root directory exists
            
            logger.info(f"Downloading data from Kaggle using dataset URL: {dataset_url}")

            # Using subprocess to call the Kaggle CLI command for dataset download
            command = f"kaggle datasets download -d maedemaftouni/large-covid19-ct-slice-dataset -p {self.config.root_dir}"
            subprocess.run(command, shell=True, check=True)

            # Move the downloaded file to the specified location
            if os.path.exists(zip_download_dir):
                logger.info(f"Downloaded data to {zip_download_dir}")
            else:
                raise FileNotFoundError(f"Failed to download dataset. File not found at {zip_download_dir}")

        except Exception as e:
            logger.error(f"An error occurred while downloading the file: {e}")
            raise e

    def extract_zip_file(self):
        """
        Extract the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")
        except zipfile.BadZipFile as e:
            logger.error(f"Failed to extract zip file: {e}")
            raise e
        except FileNotFoundError as e:
            logger.error(f"Zip file not found: {e}")
            raise e


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-07 21:49:24,520: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-08-07 21:49:24,523: INFO: common: YAML file: params.yaml loaded successfully]
[2024-08-07 21:49:24,524: INFO: common: Created directory at: artifacts]
[2024-08-07 21:49:24,526: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-08-07 21:49:24,526: INFO: 1179144871: Downloading data from Kaggle using dataset URL: kaggle datasets download -d maedemaftouni/large-covid19-ct-slice-dataset]
Dataset URL: https://www.kaggle.com/datasets/maedemaftouni/large-covid19-ct-slice-dataset
License(s): other


 98%|█████████▊| 2.02G/2.06G [00:12<00:00, 156MB/s] 


[2024-08-07 21:49:38,780: INFO: 1179144871: Downloaded data to artifacts/data_ingestion/large-covid19-ct-slice-dataset.zip]


100%|██████████| 2.06G/2.06G [00:13<00:00, 168MB/s]


[2024-08-07 21:50:01,481: INFO: 1179144871: Extracted zip file to artifacts/data_ingestion]
